# MNIST CNN Model


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, Dense, Flatten

In [3]:
NUM_EPOCHS = 10
BATCH_SIZE = 256

### Load datasets

In [4]:
train_ds, train_info = tfds.load(
    'mnist',
    split='train[10%:]',
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [5]:
val_ds, val_info = tfds.load(
    'mnist',
    split='train[0%:10%]',
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [6]:
test_ds, test_info = tfds.load(
    'mnist',
    split='test',
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

### Preprocess Data

In [7]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

def preprocess_ds(ds):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
  ds = ds.cache()
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

In [8]:
train_ds = preprocess_ds(train_ds)
val_ds = preprocess_ds(val_ds)
test_ds = preprocess_ds(test_ds)

## Create and train the model


In [9]:
model = tf.keras.models.Sequential([
    Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)),
    BatchNormalization(),

    Conv2D(32, kernel_size = 3, activation='relu'),
    BatchNormalization(),

    Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Conv2D(64, kernel_size = 3, activation='relu'),
    BatchNormalization(),

    Conv2D(64, kernel_size = 3, activation='relu'),
    BatchNormalization(),

    Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Conv2D(128, kernel_size = 4, activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dropout(0.4),
    Dense(10, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

cnn_history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=val_ds,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


211/211 [==============================] - 20s 37ms/step - loss: 0.3045 - sparse_categorical_accuracy: 0.9069 - val_loss: 3.8451 - val_sparse_categorical_accuracy: 0.3048
Epoch 2/10
211/211 [==============================] - 5s 25ms/step - loss: 0.0698 - sparse_categorical_accuracy: 0.9794 - val_loss: 1.0891 - val_sparse_categorical_accuracy: 0.7080
Epoch 3/10
211/211 [==============================] - 5s 24ms/step - loss: 0.0500 - sparse_categorical_accuracy: 0.9852 - val_loss: 0.0435 - val_sparse_categorical_accuracy: 0.9860
Epoch 4/10
211/211 [==============================] - 5s 25ms/step - loss: 0.0418 - sparse_categorical_accuracy: 0.9871 - val_loss: 0.0364 - val_sparse_categorical_accuracy: 0.9900
Epoch 5/10
211/211 [==============================] - 5s 24ms/step - loss: 0.0330 - sparse_categorical_accuracy: 0.9900 - val_loss: 0.0339 - val_sparse_categorical_accuracy: 0.9907
Epoch 6/10
211/211 [==============================] - 5s 24ms/step - loss: 0.0305 - sparse_categorical_ac

In [10]:
cnn_history.history

{'loss': [0.30451446771621704,
  0.06980445235967636,
  0.05001801624894142,
  0.04184595122933388,
  0.03302045166492462,
  0.030511081218719482,
  0.028691276907920837,
  0.024497192353010178,
  0.02220303937792778,
  0.02206232212483883],
 'sparse_categorical_accuracy': [0.9069444537162781,
  0.9794444441795349,
  0.9851666688919067,
  0.987074077129364,
  0.9900000095367432,
  0.990407407283783,
  0.9911481738090515,
  0.9923518300056458,
  0.9930740594863892,
  0.9927407503128052],
 'val_loss': [3.8450937271118164,
  1.0891399383544922,
  0.04353718087077141,
  0.03641389310359955,
  0.03388475999236107,
  0.04362216219305992,
  0.03621741756796837,
  0.030572012066841125,
  0.02968897670507431,
  0.033758603036403656],
 'val_sparse_categorical_accuracy': [0.304833322763443,
  0.7080000042915344,
  0.9860000014305115,
  0.9900000095367432,
  0.9906666874885559,
  0.9883333444595337,
  0.9904999732971191,
  0.9916666746139526,
  0.9925000071525574,
  0.9918333292007446]}

### Evaluate Model

In [12]:
result = model.evaluate(test_ds)
dict(zip(model.metrics_names, result))

40/40 [==============================] - 0s 9ms/step - loss: 0.0261 - sparse_categorical_accuracy: 0.9925


{'loss': 0.02606379985809326,
 'sparse_categorical_accuracy': 0.9925000071525574}